In [1]:
import torch
import numpy as np
import pandas as pd
import matchzoo as mz
print('matchzoo version', mz.__version__)

matchzoo version 1.2


In [2]:
ranking_task = mz.tasks.Ranking(losses=mz.losses.RankHingeLoss())
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]
print("`ranking_task` initialized with metrics", ranking_task.metrics)

`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]


In [3]:
print('data loading ...')
train_pack_raw = mz.datasets.wiki_qa.load_data('train', task=ranking_task)
dev_pack_raw = mz.datasets.wiki_qa.load_data('dev', task=ranking_task, filtered=True)
test_pack_raw = mz.datasets.wiki_qa.load_data('test', task=ranking_task, filtered=True)
print('data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`')

data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`


In [4]:
preprocessor = mz.models.MatchSRNN.get_default_preprocessor()

In [5]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 6717.92it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:04<00:00, 4015.16it/s]
Building Vocabulary from a datapack.: 100%|██████████| 418401/418401 [00:00<00:00, 2465662.81it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 7815.51it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:04<00:00, 4013.76it/s]
Processing length_right with len: 100%|██████████| 18841/18841 [00:00<00:00, 728803.40it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 7740.21it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00<00

In [6]:
preprocessor.context

{'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x7f51e147e9e8>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7f5254711e80>,
 'vocab_size': 30058,
 'embedding_input_dim': 30058}

In [7]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=100)
term_index = preprocessor.context['vocab_unit'].state['term_index']
embedding_matrix = glove_embedding.build_matrix(term_index)
l2_norm = np.sqrt((embedding_matrix * embedding_matrix).sum(axis=1))
embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [8]:
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=2,
    num_neg=1,
    batch_size=20,
    resample=True,
    sort=False
)
testset = mz.dataloader.Dataset(
    data_pack=test_pack_processed,
    batch_size=20
)

In [9]:
padding_callback = mz.models.MatchSRNN.get_default_padding_callback()

trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    stage='train',
    callback=padding_callback
)
testloader = mz.dataloader.DataLoader(
    dataset=testset,
    stage='dev',
    callback=padding_callback
)

In [10]:
model = mz.models.MatchSRNN()

model.params['task'] = ranking_task
model.params['embedding'] = embedding_matrix
model.params['channels'] = 4
model.params['units'] = 10
model.params['dropout'] = 0.2
model.params['direction'] = 'lt'

model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

MatchSRNN(
  (embedding): Embedding(30058, 100, padding_idx=0)
  (dropout): Dropout(p=0.2, inplace=False)
  (matching_tensor): MatchingTensor()
  (spatial_gru): SpatialGRU(
    (_activation): Tanh()
    (_recurrent_activation): Sigmoid()
    (_wr): Linear(in_features=34, out_features=30, bias=True)
    (_wz): Linear(in_features=34, out_features=40, bias=True)
    (_w_ij): Linear(in_features=4, out_features=10, bias=True)
    (_U): Linear(in_features=30, out_features=10, bias=False)
  )
  (out): Linear(in_features=10, out_features=1, bias=True)
)
Trainable params:  3048611


In [11]:
optimizer = torch.optim.Adadelta(model.parameters())

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=10
)

In [12]:
trainer.run()

[Iter-102 Loss-0.991]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3639 - normalized_discounted_cumulative_gain@5(0.0): 0.4344 - mean_average_precision(0.0): 0.4075



[Iter-204 Loss-0.741]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3204 - normalized_discounted_cumulative_gain@5(0.0): 0.3923 - mean_average_precision(0.0): 0.3734



[Iter-306 Loss-0.500]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.343 - normalized_discounted_cumulative_gain@5(0.0): 0.423 - mean_average_precision(0.0): 0.3822



[Iter-408 Loss-0.336]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3743 - normalized_discounted_cumulative_gain@5(0.0): 0.4479 - mean_average_precision(0.0): 0.4121



[Iter-510 Loss-0.214]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3219 - normalized_discounted_cumulative_gain@5(0.0): 0.4064 - mean_average_precision(0.0): 0.3789



[Iter-612 Loss-0.143]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3246 - normalized_discounted_cumulative_gain@5(0.0): 0.4139 - mean_average_precision(0.0): 0.3728



[Iter-714 Loss-0.110]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3284 - normalized_discounted_cumulative_gain@5(0.0): 0.3923 - mean_average_precision(0.0): 0.3693



[Iter-816 Loss-0.086]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3154 - normalized_discounted_cumulative_gain@5(0.0): 0.398 - mean_average_precision(0.0): 0.3637



[Iter-918 Loss-0.061]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3455 - normalized_discounted_cumulative_gain@5(0.0): 0.4325 - mean_average_precision(0.0): 0.3915



[Iter-1020 Loss-0.059]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3373 - normalized_discounted_cumulative_gain@5(0.0): 0.4199 - mean_average_precision(0.0): 0.3908

Cost time: 2030.2183501720428s
